# Типы в Julia

- **Примитивный тип**: тип, определяемый с помощью ключевого слова `primitive type`. Объекты примитивного типа имеют заданный фиксированный размер памяти, указанный в определении типа. 📝`Int64`,`Bool`,`Char`

- **Составной тип**: тип, определяемый с помощью ключевого слова `struct`. Составные типы состоят из нуля или более полей, ссылающихся на другие объекты (примитивного или составного типа).📝`Complex`,`Rational` (поля `re, im` и `num, den`, соответственно), `Tuple`
- **Конкретный тип**: примитивный или составной тип

- **Абстрактный тип**: тип, определяемый с помощью ключевого слова `abstract type`. Абстрактные типы не имеют полей, и объекты не могут быть созданы (инстанцированы) на их основе. Кроме того, они не могут быть объявлены дочерними по отношению к конкретному типу. Также к абстрактным типам относятся не конкретные типы.📝 `Number`, `AbstractFloat`

- **Изменяемый тип**: составной тип, определяемый с помощью ключевого слова `mutable struct`. Изменяемые типы могут связывать свои поля с другими объектами, отличными от тех, с которыми они были связаны во время инициализации.📝 `String`, `Dict`

- **Неизменяемый тип**: все типы, кроме тех, которые определяются с помощью `mutable struct`.

- **Параметрический тип**: семейство (изменяемых или неизменяемых) составных или абстрактных типов с одинаковыми именами полей и названием типа без учёта типов параметров. Определённый тип затем однозначно идентифицируется по имени параметрического типа и типу (типам) параметра (параметров). 📝  `Rational{Int8}(1,2)`, см. ниже `AbstractArray{T,N}`, `Array{T,N}`

- **Исходные типы**: тип, определение которого содержится в Julia Base или в стандартной библиотеке Julia
- **Битовый тип**: примитивный или неизменяемый составной тип, все поля которого являются битовыми типами 

- **Синглтон**: объект, созданный на основе составного типа, состоящего из нуля полей. 📝`Nothing`

- **Контейнер**: составной тип (не обязательно изменяемый), предназначенный для ссылки на переменное количество объектов и предоставляющий методы для доступа, перебора и, в конечном итоге, изменения ссылок на другие объекты. 

### Примитивный тип

In [1]:
primitive type FilledBool  <: Signed  8 end

FilledBool(x::Int8) = iszero(x) ? reinterpret(FilledBool,Int8(0)) : reinterpret(FilledBool,Int8(-1))
Int8(x :: FilledBool) = reinterpret(Int8, x) * -1

Base.show(io :: IO, x :: FilledBool) = print(io, bitstring(x))

@show tr = FilledBool(Int8(10))
@show fls = FilledBool(Int8(0))
println("Reguale Bool true: ", bitstring(true))

Int8(tr)

tr = FilledBool(Int8(10)) = 11111111
fls = FilledBool(Int8(0)) = 00000000
Reguale Bool true: 00000001


1

In [2]:
isbitstype(FilledBool)

true

### Составной тип

### Неизменяемый составной тип

In [3]:
struct Mountain
    first_ascent_year::Int16
    height::UInt16
end

Everest = Mountain(1953,8848)
Int(Everest.height)

try
    Everest.height = 9000  # нельзя менять значения полей Mountain
catch e 
e
end

ErrorException("setfield!: immutable struct of type Mountain cannot be changed")

Каждый тип элемента *неизменяемой структуры* Mountain является *битовым*, поэтому тип Mountain  является *битовым*

In [4]:
@show sizeof(Mountain) # 2 поля по 2 байта = 4
isbitstype(Mountain)

sizeof(Mountain) = 4


true

Рассмотрим случай, когда полями неизменяемой структуры является не битовый тип.

Строка хранится не как массив элементов `Char`'ов, а как указатель на массив  `Char`'ов.
Поэтому размер структуры - 8 байт (размер указателя), а размер строки - 6 байт.
(Хотя `sizeof(Char)=4`, в случае ASCII они будут занимать 1 байт)

In [5]:
struct City
    name::String
end

Moscow = City("Moscow")

Moscow.name

@show sizeof(Moscow)
@show sizeof(Moscow.name)
@show Base.summarysize(Moscow);

sizeof(Moscow) = 8
sizeof(Moscow.name) = 6
Base.summarysize(Moscow) = 22


Если требуется использовать статические строки, то

In [6]:
using StaticStrings
struct StaticCity
    name::StaticString{10}
end
Moscow = StaticCity(static"Moscow"10) # дополняется \0 до 10
@show sizeof(Moscow)
@show sizeof(Moscow.name)
@show Base.summarysize(Moscow);

sizeof(Moscow) = 10
sizeof(Moscow.name) = 10
Base.summarysize(Moscow) = 10


Несмотря на то, что мы не можем изменить строку, этот тип не является битовым.

Т.е. важно понимать отличие между *неизменяемым* и *битовым* типами.

Необычное поведение функции ismutable("123") объясняется [здесь](https://github.com/JuliaLang/julia/pull/47616)

In [7]:
@show isbitstype(City)
@show isbitstype(StaticCity);

isbitstype(City) = false
isbitstype(StaticCity) = true


Хочется отдельно отметить, что неизменяемый тип может иметь неизменяемые поля изменяемого типа.

В качестве аналогии:
Пускай у нас есть верёвочка, к которой привязан жёлтый мяч, который мы можем изменять: растягивать, завязывать в узел.
Но мы не можем оторвать верёвочку и прикрепить к ней зелёный мяч.

In [8]:
struct Student
    name::String
    grade::UInt8        # класс
    grades::Vector{Int} # оценки
 end
 Alex = Student("Alex", 1, [5,5,5])
 @show sizeof(Alex)  # 8 + 1 + 8 = 17 => 24 округление до x % 8 == 0

sizeof(Alex) = 24


24

Как можем заметить - мы меняем элементы вектора, но не указатель на его первый элемент.

In [9]:
# разыменование указателя на вектор (1й элемент вектора)
unsafe_load(pointer(Alex.grades)) 

5

А если же мы захотим поменять не элементы вектора, а указатель на вектор, то произойдёт ошибка. 

In [10]:
@show pointer(Alex.grades)
@show push!(Alex.grades, 4)
@show pointer(Alex.grades)
try
Alex.grades = [1, 2, 3] # здесь же мы хотим 
catch e
    e
end

pointer(Alex.grades) = Ptr{Int64} @0x00000290d7ef3740
push!(Alex.grades, 4) = [5, 5, 5, 4]
pointer(Alex.grades) = Ptr{Int64} @0x00000290d849b7b0


ErrorException("setfield!: immutable struct of type Student cannot be changed")

### Изменяемый тип

В случае же изменяемого типа мы можем менять поля.

In [11]:
mutable struct MutableStudent
    const name::String
    grade::UInt8        # класс
    grades::Vector{Int} # оценки
end
Peter = MutableStudent("Peter", 1, [5,5,5])
Peter.grade = 2

2

Но есть возможность делать некоторые поля изменяемой структуры неизменяемыми (константными).
В этом случае, несмотря на то, что структура - изменяемая, это поле не получится поменять.

In [12]:
try
    Peter.name = "Alex"
catch e
    e
end

ErrorException("setfield!: const field .name of type MutableStudent cannot be changed")

Можно заметить, как  теперь мы можем менять вектор на другой:

In [13]:
@show pointer(Peter.grades)
@show Peter.grades = [2,2,2]
@show pointer(Peter.grades)

pointer(Peter.grades) = Ptr{Int64} @0x00000290d8609620
Peter.grades = [2, 2, 2] = [2, 2, 2]
pointer(Peter.grades) = Ptr{Int64} @0x00000290d8a931a0


Ptr{Int64} @0x00000290d8a931a0

### Отличие неизменяемой struct от  mutable struct с константными полями.

Несмотря на то, что поля неизменяемой структуры и константные поля изменяемой структуры нельзя менять, есть существенная разница между объектами таких типов с одинаковыми полями. 

В случае с неизменяемым типом - объекты с одинаковыми полями это буквально один и тот же объект, так как все объекты с одинаковыми полями будут располагаться по одному адресу.

В случае с `mutable struct` каждый из объектов с одинаковыми константными полями будут располагаться по своему уникальному адресу.

In [14]:
struct Immutable
    a::Int32
    b::Int32
 end

 mutable struct ConstMutable
    const a::Int32
    const b::Int32
end

im_obj_1 = Immutable(1,2)
im_obj_2 = Immutable(1,2)

const_mut_obj_1 = ConstMutable(1,2)
const_mut_obj_2 = ConstMutable(1,2)
# === означает равенство и значений и адресов в памяти
@show im_obj_1 === im_obj_2  
@show const_mut_obj_1 === const_mut_obj_2

im_obj_1 === im_obj_2 = true
const_mut_obj_1 === const_mut_obj_2 = false


false

Неизменяемые структуры могут быть не такие удобные в плане интерфейса их использования.
Но их преимуществом является размещение "на стеке". В то время как изменяемые структуры **обычно** хранятся "в куче".


In [15]:
println(@allocations (a = Immutable(3,4); b = Immutable(3,4)))
println(@allocations (a = ConstMutable(3,4); b = ConstMutable(3,4)))

0
2


Однако к этому утверждению [не нужно относиться буквально](https://discourse.julialang.org/t/why-mutable-structs-are-allocated-on-the-heap/12992/27?u=garazha-ilya).

Так, например, компилятор может провести оптимизации и не выделять память для изменяемых структур внутри функции, которая будет возвращать число, а не изменяемую структуру:

In [16]:

function foo(x,y)
    obj1 = Immutable(x,y)
    obj2 = Immutable(y,x)
    c = obj1.a + obj2.b
end
function bar(x,y)
    obj1 = ConstMutable(x,y)
    obj2 = ConstMutable(y,x)
    c = obj1.a + obj2.b
end
println(@allocations foo(1,2))
println(@allocations bar(1,2))

0
0
